In [68]:
!pip install elasticsearch==8.9

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.5/395.5 kB 703.7 kB/s eta 0:00:0000:0100:01
  Attempting uninstall: elasticsearch
    Found existing installation: elasticsearch 8.6.2
    Uninstalling elasticsearch-8.6.2:
      Successfully uninstalled elasticsearch-8.6.2


In [2]:
pip -q install eland elasticsearch sentence_transformers transformers torch==1.11

Note: you may need to restart the kernel to use updated packages.


In [1]:
from elasticsearch import Elasticsearch, helpers
from urllib.request import urlopen
import json
from pathlib import Path
from eland.ml.pytorch import PyTorchModel
from eland.ml.pytorch.transformers import TransformerModel
from elasticsearch.client import MlClient
import getpass

/Users/priscilla/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/var/folders/5r/774p5f6x3jb5md2ycsn04blc0000gn/T/ipykernel_3557/982692874.py:7: DeprecationWarning: Importing from the 'elasticsearch.client' module is deprecated. Instead use 'elasticsearch' module for importing the client.
  from elasticsearch.client import MlClient


In [2]:
# Found in the 'Manage Deployment' page
CLOUD_ID = getpass.getpass('Enter Elastic Cloud ID:  ')

# Password for the 'elastic' user generated by Elasticsearch
ELASTIC_PASSWORD = getpass.getpass('Enter Elastic password:  ')

# Create the client instance
client = Elasticsearch(
    cloud_id=CLOUD_ID,
    basic_auth=("elastic", ELASTIC_PASSWORD),
    request_timeout=3600
)

Enter Elastic Cloud ID:   ········
Enter Elastic password:   ········


In [5]:
# Set the model name from Hugging Face and task type
# sentence-transformers model
hf_model_id='sentence-transformers/all-mpnet-base-v2'
tm = TransformerModel(hf_model_id, "text_embedding")

#set the modelID as it is named in Elasticsearch
es_model_id = tm.elasticsearch_model_id()

# Download the model from Hugging Face
tmp_path = "models"
Path(tmp_path).mkdir(parents=True, exist_ok=True)
model_path, config, vocab_path = tm.save(tmp_path)

# Load the model into Elasticsearch
ptm = PyTorchModel(client, es_model_id)
ptm.import_model(model_path=model_path, config_path=None, vocab_path=vocab_path, config=config)

# Start the model
s = MlClient.start_trained_model_deployment(client, model_id=es_model_id)
s.body

100%|█████████████████████████████████████| 104/104 [04:28<00:00,  2.58s/ parts]


{'assignment': {'task_parameters': {'model_id': 'sentence-transformers__all-mpnet-base-v2',
   'deployment_id': 'sentence-transformers__all-mpnet-base-v2',
   'model_bytes': 435655937,
   'threads_per_allocation': 1,
   'number_of_allocations': 1,
   'queue_capacity': 1024,
   'cache_size': '435655937b',
   'priority': 'normal'},
  'routing_table': {},
  'assignment_state': 'starting',
  'reason': 'No ML nodes exist in the cluster',
  'start_time': '2023-07-27T21:49:03.733791449Z',
  'max_assigned_allocations': 0}}

In [54]:
client.indices.create(
    index="ecommerce",
    mappings= {
    "properties": {
      "product": {
        "type": "text",
        "fields": {
          "keyword": {
            "type": "keyword",
            "ignore_above": 256
          }
        }
      },
      "description": {
        "type": "text",
        "fields": {
          "keyword": {
            "type": "keyword",
            "ignore_above": 256
          }
        }
      },
      "category": {
        "type": "text",
        "fields": {
          "keyword": {
            "type": "keyword",
            "ignore_above": 256
          }
        }
      }
  }
})

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'ecommerce'})

In [7]:
# Creating an ingest pipeline with inference processors to use ELSER (sparse) and all-mpnet-base-v2 (dense) to infer against data that will be ingested in the pipeline.

client.ingest.put_pipeline( 
    id="ecommerce-pipeline",
    processors = [
    {
      "inference": {
        "model_id": ".elser_model_1",
        "target_field": "ml",
        "field_map": {
          "description": "text_field"
        },
        "inference_config": {
          "text_expansion": { # text_expansion inference type (ELSER)
            "results_field": "tokens"
          }
        }
      }
    },
    {
      "inference": {
        "model_id": "sentence-transformers__all-mpnet-base-v2",
        "target_field": "description_vector", # Target field for the inference results
        "field_map": {
          "description": "text_field" # Field matching our configured trained model input. Typically for NLP models, the field name is text_field.
        }
      }
    }
  ]
)

ObjectApiResponse({'acknowledged': True})

In [57]:
INDEX = 'ecommerce-search'
client.indices.create(
            index=INDEX,
            settings={
                "index": {
                    "number_of_shards": 1,
                    "number_of_replicas": 1
                }
            },
            mappings={
# Saving disk space by excluding the ELSER tokens and the dense_vector field from document source.
# Note: That should only be applied if you are certain that reindexing will not be required in the future.
            "_source" : {
            "excludes": ["ml.tokens","description_vector.predicted_value"]
           }, 
        "properties": {
        "product": {
        "type": "text",
        "fields": {
          "keyword": {
            "type": "keyword",
            "ignore_above": 256
          }
        }
      },
      "description": {
        "type": "text",
        "fields": {
          "keyword": {
            "type": "keyword",
            "ignore_above": 256
          }
        }
      },
      "category": {
        "type": "text",
        "fields": {
          "keyword": {
            "type": "keyword",
            "ignore_above": 256
          }
        }
      },
      "ml.tokens": { # The name of the field to contain the generated tokens.
        "type": "rank_features" # ELSER output must be ingested into a field with the rank_features field type.
      }, 
     "description_vector.predicted_value": { # Inference results field, target_field.predicted_value
     "type": "dense_vector", 
     "dims": 768, # The all-mpnet-base-v2 model has embedding_size of 768, so dims is set to 768.
     "index": "true", 
     "similarity": "dot_product" #  When indexing vectors for approximate kNN search, you need to specify the similarity function for comparing the vectors.
 }
  }
 
}
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'ecommerce-search'})

In [58]:
# Ecommerce dataset
url = "https://raw.githubusercontent.com/priscillaparodi/dataset/main/products-ecommerce.json"

response = urlopen(url)

# Load the response data into a JSON object
data_json = json.loads(response.read())

def create_index_body(doc):
    """ Generate the body for an Elasticsearch document. """
    return {
        "_index": "ecommerce",
        "_source": doc,
    }

# Prepare the documents to be indexed
documents = [create_index_body(doc) for doc in data_json]

# Use helpers.bulk to index
helpers.bulk(client, documents)

print("Done indexing documents into `ecommerce` index")

Done indexing documents into `ecommerce` index


In [59]:
# Reindex data from one index 'source' to another 'dest' with the 'ecommerce-pipeline' pipeline.

client.reindex(wait_for_completion=True,
               source={
                  "index": "ecommerce"
    },
               dest= {
                  "index": "ecommerce-search",
                  "pipeline": "ecommerce-pipeline"
    }
)

ObjectApiResponse({'took': 18181, 'timed_out': False, 'total': 2495, 'updated': 0, 'created': 2495, 'deleted': 0, 'batches': 3, 'version_conflicts': 0, 'noops': 0, 'retries': {'bulk': 0, 'search': 0}, 'throttled_millis': 0, 'requests_per_second': -1.0, 'throttled_until_millis': 0, 'failures': []})

In [70]:
# List of results

# BM25

print(f"\nBM25:\n")

response1 = client.search(size=3,
    index="ecommerce-search",
    query= {
            "match": {
                    "description" : {  
                    "query": "Drum for beginners"
                        }
                    }
        }
)
hits = response1['hits']['hits']

if not hits:
    print("BM25 Result: No matches found")
else:
    for hit in hits:
        score = hit['_score']
        product = hit['_source']['product']
        category = hit['_source']['category']
        description = hit['_source']['description']
        print(f"\nScore: {score}\nProduct: {product}\nCategory: {category}\nDescription: {description}\n")

# KNN

print(f"\nKNN:\n")

response2 = client.search(index='ecommerce-search', size=3,
            knn={
            "field": "description_vector.predicted_value",
            "k": 49,
            "num_candidates": 2495,
            "query_vector_builder": {
            "text_embedding": { 
            "model_id": "sentence-transformers__all-mpnet-base-v2", 
            "model_text": "A drum for someone who is beginning to play." 
      }
    }
            }
)

for hit in response2['hits']['hits']:
    
    score = hit['_score']
    product = hit['_source']['product']
    category = hit['_source']['category']
    description = hit['_source']['description']
    print(f"\nScore: {score}\nProduct: {product}\nCategory: {category}\nDescription: {description}\n")

# ELSER

print(f"\nELSER:\n")

response3 = client.search(index='ecommerce-search', size=3,
              query={
                  "text_expansion": {
                  "ml.tokens": {
                      "model_id":".elser_model_1",
                      "model_text":"A drum for someone who is beginning to play."                
        }
    }
}
)

for hit in response3['hits']['hits']:

    score = hit['_score']
    product = hit['_source']['product']
    category = hit['_source']['category']
    description = hit['_source']['description']
    print(f"\nScore: {score}\nProduct: {product}\nCategory: {category}\nDescription: {description}\n")

print(f"\nHybrid Search\n")

# BM25 + KNN

print(f"\nBM25 + KNN:\n")

response4 = client.search(index='ecommerce-search', size=3,
              query={
             "match": {
             "description" : {  
             "query": "Drum for beginners",
              "boost": 1
                        }
                        }                 
                   
},
            knn={
            "field": "description_vector.predicted_value",
            "k": 49,
            "num_candidates": 2495,
            "boost": 1,
            "query_vector_builder": {
            "text_embedding": { 
            "model_id": "sentence-transformers__all-mpnet-base-v2", 
            "model_text": "A drum for someone who is beginning to play" 
      }
    }
            }
)

for hit in response4['hits']['hits']:

    score = hit['_score']
    product = hit['_source']['product']
    category = hit['_source']['category']
    description = hit['_source']['description']
    print(f"\nScore: {score}\nProduct: {product}\nCategory: {category}\nDescription: {description}\n")

# BM25 + KNN (RRF)

print(f"\nBM25 + KNN (RRF):\n")

response5 = client.search(index='ecommerce-search', size=3,
             query = {
             "match": {
             "description" : {  
             "query": "Drum for beginners",
                        }
                        }                 
                   
},
             knn = {
            "field": "description_vector.predicted_value",
            "k": 49,
            "num_candidates": 2495,
            "query_vector_builder": {
            "text_embedding": { 
            "model_id": "sentence-transformers__all-mpnet-base-v2", 
            "model_text": "A drum for someone who is beginning to play" 
      }
    }
            },
        rank = {
        "rrf": {
            "window_size": 2495,
            "rank_constant": 10
        }
    }
)

for hit in response5['hits']['hits']:
    
    rank = hit['_rank']
    category = hit['_source']['category']
    product = hit['_source']['product']
    description = hit['_source']['description']
    print(f"\nRank: {rank}\nProduct: {product}\nCategory: {category}\nDescription: {description}\n")

# BM25 + ELSER
# Note: Client is not compatible with Elastic 8.9 to use sub_searches parameter. Not adding RRF, for now.
# Using compound query - 'should' appear in the matching document.

print(f"\nBM25 + ELSER:\n")

response6 = client.search(index='ecommerce-search', size=3,

        query= {
            "bool": {
                "should": [
                    {
                        "match": {
                            "description" : {  
                            "query": "Drum for beginners",
                            "boost": 1
                        }
                        }
                    },                   
                    {
                        "text_expansion": {
                            "ml.tokens": {
                                "model_id": ".elser_model_1",
                                "model_text": "A drum for someone who is beginning to play",
                                "boost": 1
                            }
                        }
                    }
                ]
            }
        }

)

for hit in response6['hits']['hits']:

    score = hit['_score']
    product = hit['_source']['product']
    category = hit['_source']['category']
    description = hit['_source']['description']
    print(f"\nScore: {score}\nProduct: {product}\nCategory: {category}\nDescription: {description}\n")


BM25:


Score: 7.8432913
Product: Roland TD-17KVX V-Drums Electronic Drum Kit
Category: Musical Instruments
Description: is a high-quality electronic drum kit with mesh-head pads, realistic sounds, and a responsive drum module. It is perfect for practice and performance.


Score: 7.459086
Product: Dremel Digilab 3D20 3D Printer
Category: 3D Printers and Accessories
Description: is a reliable and compact 3D printer, perfect for educational use and beginners.


Score: 7.2666483
Product: Black Diamond Momentum Climbing Harness
Category: Sports Equipment
Description: is a comfortable and lightweight climbing harness designed for both beginners and experienced climbers.


KNN:


Score: 0.80521715
Product: Pearl Roadshow RS525SC Drum Set
Category: Musical Instruments
Description: is an entry-level drum set with poplar shells and double-braced hardware. It is perfect for beginner drummers.


Score: 0.7987119
Product: Roland TD-17KVX V-Drums Electronic Drum Kit
Category: Musical Instruments
D